In [1]:
!uv pip install model2vec scikit-learn

import pandas as pd
from model2vec import StaticModel
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

model = StaticModel.from_pretrained("minishlab/potion-base-8M")
df=pd.read_json("dfoeufs_with_predictions_with_ground_truth.jsonl", lines=True).dropna(subset="ocr_text").set_index("code")
dfgt=df[df.ground_truth != 'None'  ].dropna(subset="ground_truth")
dfgt


Audited 2 packages in 78ms


,lewagon_prediction,keras_category_classifier_v3_prediction,ocr_text,ground_truth
code,,,,
1.660100e+05,"{'proba_1': 'sweets', 'confidence_1': 0.0397, ...","[[en:free-range-chicken-eggs, 0.45007637143135...",British\nLion\nQuality\nSTORAGE\nFor Best Befo...,free_range
2.154730e+05,"{'proba_1': 'biscuits and cakes', 'confidence_...","[[en:barn-chicken-eggs, 0.324290543794631], [e...",15\nSERVINGS\n15 British Bar eggs. Class A. Co...,barn
5.901430e+05,"{'proba_1': 'biscuits and cakes', 'confidence_...","[[en:free-range-chicken-eggs, 0.52018284797668...",#SELL BY\nOne Dozen Large\nGrade A Brown Eggs\...,free_range
6.171901e+10,"{'proba_1': 'eggs', 'confidence_1': 0.0629, 'p...",None,REST BEFORE MEILLEUR AVANT KEEP REFRIGERATED G...,free_range
6.171901e+10,"{'proba_1': 'eggs', 'confidence_1': 0.0674, 'p...",None,▲ BEST BEFORE-MEILLEUR AVANT KEEP REFRIGERATED...,barn
...,...,...,...,...
6.749590e+11,"{'proba_1': 'eggs', 'confidence_1': 0.0545, 'p...",None,BEST BEFORE\nJN28E127\nV BEST BEFORE MEILLEUR ...,conventional_cage
4.056490e+12,"{'proba_1': 'sweets', 'confidence_1': 0.0434, ...",None,82222-00-2729\nר.II OOT P\nFOOD SNIVINO\nCORE ...,free_range
9.337716e+12,"{'proba_1': 'biscuits and cakes', 'confidence_...","[[en:free-range-chicken-eggs, 0.80595594644546...",CERTIFIED BY\nCALITY\nSUPP\nMOORABOOL VALLEY E...,free_range


In [2]:
%%time
X, y = model.encode(dfgt.ocr_text.to_list()), dfgt.ground_truth.values

CPU times: user 3.68 s, sys: 58.8 ms, total: 3.74 s
Wall time: 1.02 s


In [6]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, )
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128, 64, 32, 16, 8))
clf = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(64, 16), max_iter=int(1e7))
clf.fit(X_train, y_train)

CPU times: user 7min, sys: 505 ms, total: 7min 1s
Wall time: 56 s


MLPClassifier(hidden_layer_sizes=(64, 16), max_iter=10000000, solver='lbfgs')

In [8]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

             barn       0.80      0.84      0.82       116
conventional_cage       0.25      0.20      0.22         5
       free_range       0.96      0.93      0.95       444
   furnished_cage       0.60      0.75      0.67        24

         accuracy                           0.90       589
        macro avg       0.65      0.68      0.66       589
     weighted avg       0.91      0.90      0.90       589



In [7]:
MLPClassifier?

Init signature:
MLPClassifier(
    hidden_layer_sizes=(100,),
    activation='relu',
    *,
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    power_t=0.5,
    max_iter=200,
    shuffle=True,
    random_state=None,
    tol=0.0001,
    verbose=False,
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    n_iter_no_change=10,
    max_fun=15000,
)
Docstring:     
Multi-layer Perceptron classifier.

This model optimizes the log-loss function using LBFGS or stochastic
gradient descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden laye